In [1]:
import os
import json
import math
import numpy as np 
import time

import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from matplotlib.colors import to_rgb
import matplotlib
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms

import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

from tqdm.notebook import tqdm

import urllib.request
from urllib.error import HTTPError


# Init and Downloading

In [2]:
%matplotlib inline 
matplotlib.rcParams['lines.linewidth'] = 2.0
sns.reset_orig()
sns.set()

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = "./tmp_dataset"
os.makedirs(DATASET_PATH, exist_ok=True)
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "./tmp_saved_models/"
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cpu")
print(device)

Seed set to 42


cpu


In [3]:
# Github URL where saved models are stored for this tutorial
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial7/"
# Files to download
pretrained_files = ["NodeLevelMLP.ckpt", "NodeLevelGNN.ckpt", "GraphLevelGraphConv.ckpt"]

# For each file, check whether it already exists. If not, try downloading it.
for file_name in pretrained_files:
    file_path = os.path.join(CHECKPOINT_PATH, file_name)
    if "/" in file_name:
        os.makedirs(file_path.rsplit("/",1)[0], exist_ok=True)
    if not os.path.isfile(file_path):
        file_url = base_url + file_name
        print(f"Downloading {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
        except HTTPError as e:
            print("Something went wrong. Please try to download the file from the GDrive folder, or contact the author with the full output including the following error:\n", e)

# Introduction to Graphs

Expressing a graph as a **list of edges** is **more efficient** in terms of memory and (possibly) computation, but using an adjacency matrix simpler to implement.

_We will implement the adjacency matrix_ to keep the code simple.

Alternatively, we could also use the list of edges to _define a sparse adjacency matrix_ with which we can work as if it was a dense matrix, but allows more memory-efficient operations.


# Graph Convolutions

GCNs are similar to convolutions in images in the sense that the “filter” parameters are typically shared over all locations in the graph.
At the same time, GCNs rely on message passing methods, which means that vertices exchange information with the neighbors, and send “messages” to each other.
